In [ ]:
%cd ..

In [ ]:
from openai import OpenAI
import json
from random import seed,shuffle
from time import sleep

In [ ]:
from ragchat.html_cleaner import HtmlCleaner
from ragchat.html_patterns import HtmlPatterns


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain_community.vectorstores import FAISS


In [ ]:
cleaner = HtmlCleaner(parser="inscriptis", debug=True, max_pages=10)
clean_text_dict = cleaner.get_clean_text_dict()


In [ ]:
l_lens = []
c_lens = []
for fname in list(clean_text_dict.keys()):
    cleaned = clean_text_dict[fname]["cleaned"]
    c_lens.append((len(cleaned), fname))
    l_lens.append((len(cleaned.splitlines()), fname))

c_lens.sort()
l_lens.sort()


In [ ]:
pg = c_lens[len(c_lens) // 2][1]
text = clean_text_dict[pg]["cleaned"]
title = clean_text_dict[pg]["title"]


In [ ]:
N = len(c_lens)
docs = []
for idx in range(N):
    if N > 10:
        break
    pg = c_lens[idx][1]
    text = clean_text_dict[pg]["cleaned"]
    title = clean_text_dict[pg]["title"]
    # all_text=clean_text_dict[pg]['all_text']
    docs.append(
        Document(
            page_content=f"Title: {title}\n\n\n{text}",
            metadata={"page": pg, "title": title},
        )
    )


In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")  # for after gpt3


In [ ]:
vectorstore = FAISS.from_documents(docs, embeddings)


In [ ]:
vectorstore.save_local("vector_stores/faiss_test_0")


In [ ]:
doc_matches = vectorstore.similarity_search_with_score(
    "who is eligible for an educational license?", k=5
)
len(doc_matches)


In [ ]:
doc_matches = vectorstore.similarity_search_with_score(
    "what is the latest release for Maya?", k=150
)
len(doc_matches)


In [ ]:
vs = FAISS.load_local("vector_stores/faiss_index_0", embeddings)
doc_matches = vs.similarity_search_with_score(
    "who is eligible for an educational license?"
)


In [ ]:
[d[0].metadata for d in doc_matches]


## 